In [2]:
import csv
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Flatten,Dense,Dropout
from keras.layers import Lambda
from keras import optimizers
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

/home/jayrodge/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
lines=[]
with open('data/driving_log.csv') as csvfile:
	reader = csv.reader(csvfile)
	for line in reader:
		lines.append(line)

In [4]:
images=[]
measurements=[]
for line in lines[1:]:
	for i in range(3):
		source_path=line[i]
		filename=source_path.split('/')[-1]
		current_path='data/IMG/'+filename
		image = cv2.imread(current_path)
		images.append(image)
	measurement = float(line[3])
	correction=0.2
	measurements.append(measurement)
	measurements.append(measurement+correction)
	measurements.append(measurement-correction)

In [5]:
image_flipped=[]
measurement_flipped=[]
for i in range(len(images)):
    image_flipped.append(np.fliplr(images[i]))
    measurement_flipped.append(-1.0 * measurements[i])

In [6]:
images=images+image_flipped
measurements=measurements+measurement_flipped
del image_flipped
del measurement_flipped

In [7]:
X_train=np.array(images)
y_train=np.array(measurements)
print("Done")
model=Sequential()
#model.add(Flatten(input_shape=(160,320,3)))
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Convolution2D(6, 5, 5,activation='relu'))
#model.add(Activation("relu"))
print('Convolute')
model.add(MaxPooling2D())
model.add(Convolution2D(6, 5, 5,activation='relu'))
#model.add(Activation("relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(60))
model.add(Dropout(0.2))
# model.add(Dense(30))
# model.add(Dropout(0.3))
model.add(Dense(10))
#model.add(Dropout(0.2))
model.add(Dense(1))
print('Model Done')
#sgd = optimizers.SGD(lr=0.01)
model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])

Done


/home/jayrodge/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
  import sys
/home/jayrodge/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(6, (5, 5), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()


Convolute
Model Done


In [8]:
model.fit(x=X_train,y=y_train,nb_epoch=3,validation_split=0.2,shuffle=True)
model.save('model.h5')

/home/jayrodge/anaconda3/lib/python3.6/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 38572 samples, validate on 9644 samples
Epoch 1/3
 8352/38572 [=====>........................] - ETA: 6:20 - loss: 0.0704 - acc: 0.1771

KeyboardInterrupt: 